<a href="https://colab.research.google.com/github/enesbol/Lexicon_Extension/blob/main/Lexicon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import gensim.downloader

In [2]:
model = gensim.downloader.load('glove-twitter-100')

In [3]:
!python -m spacy download en_core_web_lg

2023-06-14 15:03:58.919768: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [4]:
# nlp = spacy.load("en_core_web_md")  # Load medium-sized English model
nlp = spacy.load("en_core_web_lg")

In [5]:
lexicon_df = pd.read_excel("/content/Consumer_Adjective_Clean.xlsx")

# trailing whitespace in column names in lexicon_df
lexicon_df.columns = lexicon_df.columns.str.strip()

In [6]:
base_vector_dict = {}

#Finally, list(dict.fromkeys(...)) is used to remove duplicates from the resulting list, ensuring only unique values are present in string_list.
for col in lexicon_df:
    # Create a list of unique lowercase, stripped, and asterisk-free values from the column
    string_list = list(dict.fromkeys([re.sub(r'[^a-zA-Z]', '', str(x).lower().strip().replace('*', '')) for x in lexicon_df[col].dropna().tolist()]))

    # Convert the string list into a NumPy array of spaCy word vectors
    string_list_vectors = np.array([nlp(s).vector for s in string_list])

    # Add the column name and its corresponding array of word vectors to the base_vector_dict
    base_vector_dict[col + "_base_vector"] = string_list_vectors

In [7]:
base_vector_dict

{'Spatial_base_vector': array([[-0.068492,  5.7971  , -0.71815 , ..., -1.8815  , -2.7432  ,
          2.6032  ],
        [-4.5671  , -0.72469 ,  0.35116 , ..., -1.8325  , -0.47618 ,
          2.9645  ],
        [-0.79739 , -2.5883  , -1.9291  , ...,  1.206   ,  0.89389 ,
         -0.142   ],
        ...,
        [ 1.5942  ,  2.4367  ,  2.4225  , ..., -1.5043  , -1.0966  ,
          1.6429  ],
        [-4.3635  ,  0.42556 , -2.5431  , ..., -5.0893  , -0.29509 ,
          1.7555  ],
        [-3.4736  ,  1.2572  ,  0.037179, ...,  0.12902 , -2.187   ,
          0.64316 ]], dtype=float32),
 'Normative_base_vector': array([[-0.41948 , -0.056428, -1.8163  , ...,  0.78343 , -4.5137  ,
          0.66713 ],
        [ 0.73066 ,  0.58313 ,  2.6591  , ..., -0.1572  , -4.7668  ,
         -0.59006 ],
        [ 1.1006  , -1.0492  , -2.8242  , ...,  1.4446  , -2.2498  ,
          0.48874 ],
        ...,
        [ 0.21383 ,  1.1599  , -3.1271  , ..., -1.4491  , -6.6107  ,
          1.4956  ],
        [

In [8]:
from tqdm import tqdm
tqdm.pandas()

# Create empty lists for potential words and out-of-vocabulary words
potential_words = []
out_words = []

# Create an empty DataFrame to store potential words
potential_words_df = pd.DataFrame()

# Iterate over each column in the lexicon_df
for col in tqdm(lexicon_df.columns):
    # Clear the potential_words list for each column
    potential_words = []

    # Clean and preprocess the words in the column
    sp = list(dict.fromkeys([re.sub(r'[^a-zA-Z]', '', str(x).lower().strip().replace('*', '')) for x in lexicon_df[col].dropna().tolist()]))

    # Iterate over each word in the cleaned list
    for word in sp:
        try:
            # Retrieve similar words for the current word using the pre-trained model.
            # Increase topn parameter to 100 for producing 100 words for each word at lexicon.
            similar_words = model.most_similar(word, topn=25)
            potential_words.append(similar_words)
        except KeyError:
            # If the word is not found in the model's vocabulary, add it to the out_words list
            out_words.append(word)

    # Extract only the words from the potential_words list and filter out non-Latin characters
    words_only = [word for sublist in potential_words for word, _ in sublist]
    latin_words = [word for word in words_only if re.match(r'^[a-zA-Z]+$', word)]
    latin_words = list(set(latin_words))

    # Add the potential words as a new column in the potential_words_df DataFrame
    potential_words_df["potential_"+col] = pd.Series(latin_words)


100%|██████████| 3/3 [00:23<00:00,  7.82s/it]


In [9]:
potential_words_df

,potential_Spatial,potential_Normative,potential_Intensity
0,kombat,societies,involves
1,abreeza,rouca,memorable
2,kingswood,guh,stunning
3,handgun,sartorial,superb
4,cave,praise,romantic
...,...,...,...
1676,persons,NaN,NaN
1677,rise,NaN,NaN
1678,axed,NaN,NaN
1679,licenses,NaN,NaN


In [10]:
# Iterate through each column in potential_words_df
for col in potential_words_df:
    potential_words_df[col+"_vectorized"] = pd.Series(potential_words_df[col].fillna('').progress_apply(lambda x: nlp(x).vector))

# Create a new column in potential_words_df with "_vectorized" suffix and populate it with the vectorized representation
# of each element in the corresponding column

100%|██████████| 1681/1681 [00:08<00:00, 202.84it/s]


In [11]:
potential_words_df.head()

,potential_Spatial,potential_Normative,potential_Intensity,potential_Spatial_vectorized,potential_Normative_vectorized,potential_Intensity_vectorized
0,kombat,societies,involves,"[1.4838, -0.83254, -0.25512, 3.2036, 1.2825, 2...","[-0.88241, -1.4953, -0.46298, 0.54697, 5.5311,...","[0.13169, 0.040969, -2.9332, -0.35453, 6.1576,..."
1,abreeza,rouca,memorable,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-1.5939, -0.59281, -1.7411, -2.8109, 3.9716, ..."
2,kingswood,guh,stunning,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.12342, 0.12828, -0.10002, -0.70007, -0.339...","[0.21545, -1.6932, -1.2753, -2.3509, 2.1391, -..."
3,handgun,sartorial,superb,"[-4.2162, 0.5107, -3.0125, 5.6097, 2.4996, 1.6...","[-1.7604, 0.14548, 1.1976, -0.062363, 2.7038, ...","[-0.16503, -1.3027, -1.7095, -1.4802, 3.1316, ..."
4,cave,praise,romantic,"[3.9985, 0.55137, 0.050358, -3.3203, -1.3749, ...","[0.55707, 0.025443, -2.7958, 0.048361, 3.9328,...","[-1.3005, -0.58608, -1.5736, -6.2532, 0.53727,..."


In [12]:
"""
The variable base_vector_dict is a dictionary that stores the base vectors for each lexicon.
In this case, base_vector_dict['Spatial_base_vector'] contains the vector representations of
all the words from the "Spatial" lexicon. These vector representations are obtained by vectorizing
the words using a pre-trained word embedding model.

To access a specific word vector, we can use indexing. In the given code, [90] is used to access the
91st word's vector representation from the "Spatial" lexicon. By calling .shape on this vector, we obtain
its shape, which in this case is (300,).

The shape (300,) indicates that the vector representation of the 91st word in the "Spatial" lexicon has a length of 300.
This means that the word is represented by a 300-dimensional vector, where each dimension captures different aspects of
the word's meaning or context.
"""

'\nThe variable base_vector_dict is a dictionary that stores the base vectors for each lexicon.\nIn this case, base_vector_dict[\'Spatial_base_vector\'] contains the vector representations of\nall the words from the "Spatial" lexicon. These vector representations are obtained by vectorizing\nthe words using a pre-trained word embedding model.\n\nTo access a specific word vector, we can use indexing. In the given code, [90] is used to access the\n91st word\'s vector representation from the "Spatial" lexicon. By calling .shape on this vector, we obtain\nits shape, which in this case is (300,).\n\nThe shape (300,) indicates that the vector representation of the 91st word in the "Spatial" lexicon has a length of 300.\nThis means that the word is represented by a 300-dimensional vector, where each dimension captures different aspects of\nthe word\'s meaning or context.\n'

In [13]:
base_vector_dict['Spatial_base_vector'][90].shape

(300,)

In [14]:
base_vector_dict['Spatial_base_vector'][90]

array([-3.4736   ,  1.2572   ,  0.037179 , -3.447    ,  2.3293   ,
        0.78382  ,  3.0362   ,  2.8004   ,  1.176    ,  0.77362  ,
       -0.13223  ,  0.65523  , -2.3919   ,  1.7062   , -0.11605  ,
       -0.62067  ,  0.090005 ,  0.41134  ,  1.1019   , -2.9679   ,
       -0.21581  , -1.6597   , -0.20205  ,  2.0588   , -0.30373  ,
       -1.2938   , -3.6779   , -1.1779   , -0.58174  ,  2.5519   ,
        0.03169  , -0.27004  , -0.086378 , -1.6005   , -4.739    ,
       -2.6656   , -1.8855   ,  2.0184   ,  0.51353  ,  0.048929 ,
       -1.3999   , -0.87185  , -0.76266  ,  2.0184   , -4.1461   ,
        2.3547   , -1.2481   , -0.89289  ,  0.17742  ,  1.9663   ,
        1.7749   ,  0.44471  , -0.79376  , -3.9409   , -0.1602   ,
        0.18716  ,  1.0368   , -2.145    ,  3.8091   ,  3.0528   ,
        0.084441 , -3.8565   ,  2.8445   , -0.89202  ,  0.10439  ,
       -0.16368  ,  1.7791   ,  0.11798  ,  2.8873   , -0.77619  ,
       -0.56284  , -2.1953   , -0.48176  ,  1.1212   ,  2.5832

In [15]:
def compute_mean_cosine_sim(row):
    # Compute the mean cosine similarity between a potential word vector and a set of base vectors

    potential_vector = row  # Store the potential word vector
    base_vectors = base_vector_dict[col.split('_')[1] + '_base_vector']  # Retrieve the corresponding base vectors

    if isinstance(base_vectors, list):
        base_vectors = np.array(base_vectors)  # Convert base vectors to NumPy array if it's a list
    if len(base_vectors) == 0:
        return 0  # Return 0 if there are no base vectors for comparison

    try:
        cos_sim = cosine_similarity(potential_vector.reshape(1, -1), base_vectors)  # Compute cosine similarity
    except:
        return 0  # Return 0 if an exception occurs during computation

    cos_sim = np.abs(cos_sim)  # Take the absolute value of cosine similarity

    return np.sum(cos_sim)  # Return the sum of cosine similarities


x = ['potential_Spatial_vectorized', 'potential_Normative_vectorized', 'potential_Intensity_vectorized']

for col in x:
    colname = col.split('_')[1] + '_score'  # Generate the score column name based on the lexicon category
    potential_words_df[colname] = potential_words_df[col].progress_apply(compute_mean_cosine_sim)

100%|██████████| 1681/1681 [00:01<00:00, 1663.91it/s]


In [16]:
potential_words_df.head()

,potential_Spatial,potential_Normative,potential_Intensity,potential_Spatial_vectorized,potential_Normative_vectorized,potential_Intensity_vectorized,Spatial_score,Normative_score,Intensity_score
0,kombat,societies,involves,"[1.4838, -0.83254, -0.25512, 3.2036, 1.2825, 2...","[-0.88241, -1.4953, -0.46298, 0.54697, 5.5311,...","[0.13169, 0.040969, -2.9332, -0.35453, 6.1576,...",12.606293,23.444916,24.400383
1,abreeza,rouca,memorable,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-1.5939, -0.59281, -1.7411, -2.8109, 3.9716, ...",0.000000,0.000000,37.044582
2,kingswood,guh,stunning,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.12342, 0.12828, -0.10002, -0.70007, -0.339...","[0.21545, -1.6932, -1.2753, -2.3509, 2.1391, -...",0.000000,5.654067,33.231323
3,handgun,sartorial,superb,"[-4.2162, 0.5107, -3.0125, 5.6097, 2.4996, 1.6...","[-1.7604, 0.14548, 1.1976, -0.062363, 2.7038, ...","[-0.16503, -1.3027, -1.7095, -1.4802, 3.1316, ...",11.293950,19.790752,31.202253
4,cave,praise,romantic,"[3.9985, 0.55137, 0.050358, -3.3203, -1.3749, ...","[0.55707, 0.025443, -2.7958, 0.048361, 3.9328,...","[-1.3005, -0.58608, -1.5736, -6.2532, 0.53727,...",24.486513,11.551569,25.790298


In [17]:
def clean_list(col):
    # Clean and preprocess the values in a column of the lexicon dataframe
    sp_list = list(dict.fromkeys([re.sub(r'[^a-zA-Z]', '', str(x).lower().strip().replace('*', '')) for x in lexicon_df[col].dropna().tolist()]))
    # Remove non-alphabetic characters, convert to lowercase, strip leading/trailing whitespace, and remove duplicates
    return sp_list

columns = ['Spatial', 'Normative', 'Intensity']
extended_lexicons = []  # List to store extended lexicons for each column

for col in columns:
    sp = clean_list(col)  # Clean the values in the column
    potential_col = 'potential_' + col  # Generate the name of the potential column
    extended_lexicon = potential_words_df[~potential_words_df[potential_col].isin(sp)].sort_values(col + '_score', ascending=False)[[potential_col, col + '_score']]
    # Filter potential words not present in the lexicon, sort by score in descending order, and select relevant columns
    extended_lexicons.append(extended_lexicon)  # Append the extended lexicon to the list

In [18]:
extended_lexicons[0].head()

,potential_Spatial,Spatial_score
1286,sheltered,33.443504
1135,underground,33.051483
312,confined,31.331417
461,surrounding,31.217823
1576,habitable,31.159948


In [19]:
sorted_extended_lexicons = []  # List to store sorted extended lexicons

for lexicon in extended_lexicons:
    sorted_lexicon = lexicon.sort_values(by=[lexicon.columns[1]], ascending=False).reset_index(drop=True)
    # Sort the lexicon DataFrame by the score column in descending order and reset the index
    sorted_extended_lexicons.append(sorted_lexicon)  # Append the sorted lexicon to the list

combined_extended_lexicon = pd.concat(sorted_extended_lexicons, axis=1)
# Concatenate the sorted extended lexicons along the columns axis to create the combined extended lexicon DataFrame

In [22]:
combined_extended_lexicon.to_excel("Result_Lexicons_with_Scores.xlsx") # save to excel.